In [1]:
import pandas as pd
import yaml
import plotly.graph_objects as go

with open("config.yml", 'r') as stream:
    CONFIG = yaml.load(stream)

# Grabbing and generating raw data (global variables)
RAW_DATA = {}

for rawfile in CONFIG['rawfiles'].keys():
    RAW_DATA[rawfile] = {"data": pd.read_csv(CONFIG['rawfiles'][rawfile], nrows=10)}
    RAW_DATA[rawfile]["all_cols"] = RAW_DATA[rawfile]["data"].columns
    RAW_DATA[rawfile]["id_cols"] = [col
                                        for col in RAW_DATA[rawfile]["data"].columns
                                            if col.lower() in [x.lower() for x in CONFIG['id_cols']]
                                    ]
# for dataset in RAW_DATA.keys():
#     print(dataset, "all cols : ", RAW_DATA[dataset]["all_cols"])
#     print(dataset, "id cols: ", RAW_DATA[dataset]["id_cols"])

def join_data(data1_name, data2_name):
    """
    Function to join two datasets together (left on data1, data2 into data1)

    Inputs
    ------

    data1_name: String - base dataset
    data2_name: String - dataset to be joined to data1

    Returns
    -------

    dict(joined_dataset, all columns of joined dataset, id columns of joined dataset)
    """
    data1 = pd.read_csv(CONFIG['rawfiles'][data1_name])
    data2 = pd.read_csv(CONFIG['rawfiles'][data2_name])
    join_keys = list(set(RAW_DATA[data1_name]['id_cols']).intersection(RAW_DATA[data2_name]['id_cols']))
    joined_data = data1.merge(data2, how='left', on=join_keys)
    return {"data": joined_data, "all_cols": joined_data.columns, "id_cols": join_keys}
    

final_data = {}
final_data['People_Batting'] = join_data('People', 'Batting')
print(final_data['People_Batting']["data"]['yearID'].value_counts(dropna=False))


2018.0    1535
2017.0    1494
2015.0    1486
2016.0    1483
2014.0    1435
2013.0    1409
2012.0    1408
2011.0    1389
2009.0    1388
2007.0    1385
2008.0    1385
2000.0    1384
2006.0    1377
2010.0    1356
2003.0    1347
2004.0    1346
2001.0    1339
2005.0    1330
1998.0    1322
2002.0    1319
1999.0    1299
1995.0    1253
1996.0    1253
1997.0    1236
1993.0    1180
1990.0    1115
1991.0    1086
1989.0    1073
1992.0    1066
1987.0    1048
          ... 
1905.0     407
1904.0     406
1903.0     400
1885.0     353
1888.0     352
1886.0     351
1899.0     348
1889.0     342
1887.0     333
1898.0     323
1892.0     312
1895.0     305
1896.0     295
1894.0     295
1893.0     286
1883.0     283
1897.0     280
1882.0     252
1875.0     218
1900.0     195
1872.0     157
1880.0     135
1881.0     132
1879.0     127
1873.0     125
1876.0     124
1874.0     123
1871.0     115
1877.0      98
1878.0      80
Name: yearID, Length: 149, dtype: int64


In [8]:
x = final_data['People_Batting']["data"]['yearID']
y = final_data['People_Batting']["data"]['yearID'].value_counts(dropna=False)
fig = go.Figure(data=go.Bar(y=y, x=x))
fig.write_html('visuals/people_batting_count_by_yr.html', auto_open=False)

In [9]:
def plot_to_html(data, x_col, y_col, filename):
    x = data[x_col]
    y = data[y_col]
    fig = go.Figure(data=go.Bar(y=y, x=x))
    fig.write_html(filename, auto_open=False)

In [12]:
plot_to_html(final_data['People_Batting']["data"], 'yearID', 'G', 'visuals/people_batting_games_by_yr.html')
plot_to_html(final_data['People_Batting']["data"], 'yearID', 'RBI', 'visuals/people_batting_rbi_by_yr.html')